Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
from math import sqrt
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
batch_size = 128
hidden_nodes = 1024

regularization_coeff = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))    
    
  def forward_prop(dataset):
        return tf.matmul(tf.nn.relu(tf.matmul(dataset, weights1) + biases1), weights2) + biases2    
  
  # Training computation.
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + \
    regularization_coeff * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 663.219604
Minibatch accuracy: 7.8%
Validation accuracy: 36.9%
Minibatch loss at step 500: 193.414185
Minibatch accuracy: 84.4%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 114.722321
Minibatch accuracy: 85.9%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 70.257652
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 41.377964
Minibatch accuracy: 86.7%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.145321
Minibatch accuracy: 91.4%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 15.520212
Minibatch accuracy: 84.4%
Validation accuracy: 86.5%
Test accuracy: 92.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 5) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 629.807373
Minibatch accuracy: 12.5%
Validation accuracy: 31.1%
Minibatch loss at step 500: 190.416382
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 115.478821
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1500: 70.032707
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 42.471733
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 25.757477
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 15.621763
Minibatch accuracy: 100.0%
Validation accuracy: 77.9%
Test accuracy: 84.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
batch_size = 128
hidden_nodes = 1024

dropout_prob = 0.75
regularization_coeff = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))    
    
  def forward_prop_w_dropout(dataset):
        hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(dataset, weights1) + biases1), dropout_prob)
        return tf.matmul(hidden_layer, weights2) + biases2

  def forward_prop_wo_dropout(dataset):
        hidden_layer = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
        return tf.matmul(hidden_layer, weights2) + biases2    
  
  # Training computation.
  logits = forward_prop_w_dropout(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + \
    regularization_coeff * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop_wo_dropout(tf_valid_dataset))
  test_prediction = tf.nn.softmax(forward_prop_wo_dropout(tf_test_dataset))

Limited train dataset with dropouts

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 5) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 654.274170
Minibatch accuracy: 9.4%
Validation accuracy: 32.3%
Minibatch loss at step 500: 190.586914
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 1000: 115.595230
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 70.110001
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 42.520267
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 25.787296
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 15.639303
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Test accuracy: 85.6%


All train datatset with dropouts

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 743.446045
Minibatch accuracy: 9.4%
Validation accuracy: 25.0%
Minibatch loss at step 500: 197.529465
Minibatch accuracy: 78.1%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 115.375900
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 70.735321
Minibatch accuracy: 78.9%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 41.727730
Minibatch accuracy: 83.6%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.156261
Minibatch accuracy: 90.6%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 15.388397
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Test accuracy: 92.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [11]:
batch_size = 128
hidden_nodes = 1024

dropout_prob = 0.75
regularization_coeff = 0.001

learning_rate = 0.5
decay_steps = 1000
learning_decay_rate = 0.98    

nn_layers = [image_size * image_size, 1024, 512, 256, num_labels]

def logits_w_dropout(first_input, weights, biases):
    input_to_next_layer = first_input
    for i in range(len(weights) - 1):
        input_to_next_layer = tf.nn.dropout(tf.nn.relu(
            tf.matmul(input_to_next_layer, weights[i]) + biases[i]), dropout_prob)
    return tf.matmul(input_to_next_layer, weights[-1]) + biases[-1]


def logits_wo_dropout(first_input, weights, biases):
    input_to_next_layer = first_input
    for i in range(len(weights) - 1):
        input_to_next_layer = tf.nn.relu(tf.matmul(input_to_next_layer, weights[i]) + biases[i])
    return tf.matmul(input_to_next_layer, weights[-1]) + biases[-1]


weights = []
biases = []
graph = tf.Graph()
with graph.as_default():
    # Constants
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)

    # Variables
    for i in range(len(nn_layers) - 1):
        weights.append(tf.Variable(tf.truncated_normal([nn_layers[i], nn_layers[i + 1]],
                                                       stddev=sqrt(1.0 / nn_layers[i]))))
        biases.append(tf.Variable(tf.zeros([nn_layers[i + 1]])))

    # Computation
    logits = logits_w_dropout(tf_train_dataset, weights, biases)
    regularization = regularization_coeff * np.sum(map(tf.nn.l2_loss, weights))
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + regularization

    # Optimizer
    global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps,
                                               learning_decay_rate, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)

    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_wo_dropout(tf_valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(logits_wo_dropout(tf_test_dataset, weights, biases))

In [12]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.002152
Minibatch accuracy: 14.1%
Validation accuracy: 30.5%
Minibatch loss at step 1000: 0.737830
Minibatch accuracy: 86.7%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 0.678053
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 0.531235
Minibatch accuracy: 88.3%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 0.535429
Minibatch accuracy: 89.8%
Validation accuracy: 87.8%
Minibatch loss at step 5000: 0.562461
Minibatch accuracy: 85.2%
Validation accuracy: 87.9%
Minibatch loss at step 6000: 0.469628
Minibatch accuracy: 90.6%
Validation accuracy: 87.9%
Minibatch loss at step 7000: 0.491343
Minibatch accuracy: 89.8%
Validation accuracy: 87.8%
Minibatch loss at step 8000: 0.483283
Minibatch accuracy: 89.8%
Validation accuracy: 88.5%
Minibatch loss at step 9000: 0.562675
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 10000: 0.388111
Minibatch accuracy: 93.0%
Validation accur